# ANPR Archivio storico dei comuni

Data processing che partendo dal dataset di [ANPR](https://www.anpr.interno.it/portale/) contenente l'Archivio storico dei comuni (02-04-2019) - xls predispone e arricchisce il dataset in modo che possa essere importato in DAF e  interoperabile con il dataset rilasciato da ISTAT.

Su DAF è presente il dataset originario creato da Alberto
https://dataportal.daf.teamdigitale.it/#/private/dataset/anpr_comuni_italiani

Sul dataset pubblicato da ANPR abbiamo aperto degli issue richiedendo che il dataset venga propriamente pubblicato seguendo le linee guida

- https://github.com/italia/anpr/issues/1252
- https://github.com/italia/anpr/issues/1253
 

Operazioni da fare pre processing:

- scraper dataset: non c'è una permament uri
- utilizzo encoding corretto
- attenzione a campi null
- il codice istat deve essere letto come string 
- campo IDPROVINCIA deve essere su tre char con padding a 0 per poter fare join con ISTAT
- formattazione campi data
- formattazione campi nome comune per denominazione in altra lingua 
- formattazione campi nome comune it e denominazione translitterata

Post processing:
- script SQL per estrarre i blocchi di comuni con denominazione in DE e FR per interlinking con Giorgia


In [124]:
import pandas as pd
import numpy as np

In [125]:
data_cessazione = '9999-12-31'

In [126]:
#fare attenzione a codice istat che deve essere letto come stringa per non perdere gli 0 in testa
# na_filter=False per non inserire NaN ma "" dove i campi sono vuoti

In [127]:
anpr = pd.read_excel("./data/tabella 3 archivio comuni 20190402.xlsx",  dtype={'CODISTAT': str}, na_filter=False)

In [128]:
anpr.dtypes

ID                          int64
DATAISTITUZIONE            object
DATACESSAZIONE             object
CODISTAT                   object
CODCATASTALE               object
DENOMINAZIONE_IT           object
DENOMTRASLITTERATA         object
ALTRADENOMINAZIONE         object
ALTRADENOMTRASLITTERATA    object
IDPROVINCIA                 int64
IDREGIONE                  object
IDPREFETTURA               object
STATO                      object
SIGLAPROVINCIA             object
FONTE                      object
DATAULTIMOAGG              object
COD_DENOM                  object
dtype: object

In [129]:
# il campo provincia è numerico ma su dataset è presentato in formato alfanumerico quindi bisogna trasformarlo
# in string e fare un padding left

In [90]:
anpr.isnull().sum()

ID                         0
DATAISTITUZIONE            0
DATACESSAZIONE             0
CODISTAT                   0
CODCATASTALE               0
DENOMINAZIONE_IT           0
DENOMTRASLITTERATA         0
ALTRADENOMINAZIONE         0
ALTRADENOMTRASLITTERATA    0
IDPROVINCIA                0
IDREGIONE                  0
IDPREFETTURA               0
STATO                      0
SIGLAPROVINCIA             0
FONTE                      0
DATAULTIMOAGG              0
COD_DENOM                  0
dtype: int64

In [91]:
anpr.head(100)

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM
0,12560,19/11/1866,13/11/1924,028001,A001,ABANO,ABANO,,,28,05,,C,PD,,17/06/2016,028500
1,1,14/11/1924,31/12/9999,028001,A001,ABANO TERME,ABANO TERME,,,28,05,PD,A,PD,,17/06/2016,
2,3,01/04/1864,05/12/1928,001801,A003,ABBADIA ALPINA,ABBADIA ALPINA,,,1,01,,C,TO,,11/11/2017,
3,5,17/03/1861,15/04/1992,015001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,15,03,,C,MI,,17/06/2016,
4,4,16/04/1992,31/12/9999,098001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,98,03,LO,A,LO,,17/06/2016,
5,6,06/03/1928,15/04/1992,013001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,13,03,,C,CO,,17/06/2016,
6,7,16/04/1992,31/12/9999,097001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,97,03,LC,A,LC,,17/06/2016,
7,8,22/07/1867,31/12/9999,052001,A006,ABBADIA SAN SALVATORE,ABBADIA SAN SALVATORE,,,52,09,SI,A,SI,,17/06/2016,
8,2,23/02/1863,06/03/1928,013801,A002,ABBADIA SOPRA ADDA,ABBADIA SOPRA ADDA,,,13,03,,C,CO,,17/06/2016,
9,11,17/03/1861,27/07/1927,092001,A007,ABBASANTA,ABBASANTA,,,92,20,,C,CA,,17/06/2016,


## Adattamento campo IDPROVINCIA 

Leggerlo come striga e fare padding con 0 su tre char

In [130]:
anpr['IDPROVINCIA'] = anpr['IDPROVINCIA'].apply(str)

In [131]:
anpr['IDPROVINCIA'] = anpr['IDPROVINCIA'].apply(lambda x: x.zfill(3))

## Adattamento formato campi data

I campi data, che al momento non sono gestiti sul form di ingestion del DAF, vanno trasformati usando il "-" come separatore.

Da fare per il dataset nuovo per il cui non sarà più necessario fare vista.

In [132]:
anpr['DATAULTIMOAGG'] = pd.to_datetime(anpr['DATAULTIMOAGG'])
anpr['DATAULTIMOAGG'] = anpr['DATAULTIMOAGG'].dt.strftime('%Y-%m-%d')

In [133]:
#su data ultimo agg bisogna ignorare errori
anpr['DATAISTITUZIONE'] = pd.to_datetime(anpr['DATAISTITUZIONE'])
anpr['DATAISTITUZIONE'] = anpr['DATAISTITUZIONE'].dt.strftime('%Y-%m-%d')

In [134]:
anpr['DATACESSAZIONE'] = pd.to_datetime(anpr['DATACESSAZIONE'], errors='coerce')

In [135]:
anpr['DATACESSAZIONE'] = anpr['DATACESSAZIONE'].dt.strftime('%Y-%m-%d')

In [ ]:
#NON FATTO
anpr['DATACESSAZIONE'].date.dt.strftime('%Y-%m-%d').replace('NaT', data_cessazione)

In [136]:
anpr['DATACESSAZIONE'] = anpr['DATACESSAZIONE'].replace('NaT', data_cessazione)

## Adattamento 
###  DENOMINAZIONE_IT e DENOMTRASLITTERATA

1) Tutte le denominazioni in italiano **translitterate** le dobbiamo trasformare in LetteraMaiuscolaLettereminuscole
così diventa confrontabile con i comuni di IPA

2) Tutte le denominazioni in italiano (non translitterate) le dobbiamo trasformare in LetteraMaiuscolaLettereminuscole. 

In presenza di doppia parola (es ABANO TERME) dobbiamo farla diventare LetteraMaiuscolaLettereminuscole_LetteraMaiuscolalettereminuscole  
queste due cose sono fondamentali anche per diversi join sui dataset IPA e per fare intelinking con DBPedia.

In [ ]:
anpr['result'] = np.where(anpr['DENOMINAZIONE_IT'] == anpr['DENOMTRASLITTERATA'], 'no change', 'changed')
print (anpr.result.value_counts())

In [ ]:
anpr[anpr['result']=='changed']

In [137]:
anpr["Denominazione_it_linking"] = anpr.DENOMINAZIONE_IT.str.title()
anpr["Denominazione_it_linking"] = anpr.Denominazione_it_linking.str.replace(" ", "_")

In [138]:
anpr["DENOMTRASLITTERATA_IPA"]=anpr.DENOMTRASLITTERATA.str.title()

In [139]:
anpr

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM,Denominazione_it_linking,DENOMTRASLITTERATA_IPA
0,12560,1866-11-19,1924-11-13,028001,A001,ABANO,ABANO,,,028,05,,C,PD,,2016-06-17,028500,Abano,Abano
1,1,1924-11-14,9999-12-31,028001,A001,ABANO TERME,ABANO TERME,,,028,05,PD,A,PD,,2016-06-17,,Abano_Terme,Abano Terme
2,3,1864-01-04,1928-05-12,001801,A003,ABBADIA ALPINA,ABBADIA ALPINA,,,001,01,,C,TO,,2017-11-11,,Abbadia_Alpina,Abbadia Alpina
3,5,1861-03-17,1992-04-15,015001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,015,03,,C,MI,,2016-06-17,,Abbadia_Cerreto,Abbadia Cerreto
4,4,1992-04-16,9999-12-31,098001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,098,03,LO,A,LO,,2016-06-17,,Abbadia_Cerreto,Abbadia Cerreto
5,6,1928-06-03,1992-04-15,013001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,013,03,,C,CO,,2016-06-17,,Abbadia_Lariana,Abbadia Lariana
6,7,1992-04-16,9999-12-31,097001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,097,03,LC,A,LC,,2016-06-17,,Abbadia_Lariana,Abbadia Lariana
7,8,1867-07-22,9999-12-31,052001,A006,ABBADIA SAN SALVATORE,ABBADIA SAN SALVATORE,,,052,09,SI,A,SI,,2016-06-17,,Abbadia_San_Salvatore,Abbadia San Salvatore
8,2,1863-02-23,1928-06-03,013801,A002,ABBADIA SOPRA ADDA,ABBADIA SOPRA ADDA,,,013,03,,C,CO,,2016-06-17,,Abbadia_Sopra_Adda,Abbadia Sopra Adda
9,11,1861-03-17,1927-07-27,092001,A007,ABBASANTA,ABBASANTA,,,092,20,,C,CA,,2016-06-17,,Abbasanta,Abbasanta


In [108]:
anpr.DATACESSAZIONE.value_counts()

9999-12-31    7915
1927-11-01    1632
1923-04-02     589
1992-04-15     368
1923-07-02     256
1968-05-04     187
1974-08-19     177
2016-04-27     155
1923-04-26     142
1970-02-03     135
2005-12-31     123
1947-09-15     118
1935-07-04     105
1945-08-31      79
1923-12-21      78
1992-05-22      77
2015-12-31      68
1923-03-18      68
1998-03-09      67
1945-04-10      63
1928-11-05      56
2009-06-29      50
1928-12-05      46
2018-12-31      45
1928-05-09      45
1928-01-25      43
2009-11-07      40
1928-12-03      40
1946-01-29      39
1928-02-20      38
              ... 
1937-10-07       1
1945-04-23       1
1956-03-11       1
1956-03-24       1
1945-11-01       1
2001-11-12       1
1954-07-23       1
2008-03-18       1
1947-01-21       1
1951-07-17       1
1916-02-03       1
1957-12-14       1
1957-07-20       1
1938-08-01       1
1934-03-13       1
1926-07-22       1
1965-01-01       1
1910-03-23       1
1938-10-02       1
1962-02-07       1
1925-01-16       1
1923-05-31  

## Salvare dataset finale con separatore e senza index

In [141]:
anpr.to_csv("./output_new_ipa.csv", index=False, sep=";")